<a href="https://colab.research.google.com/github/emisoft-designs/Data-Science-Practice-Projects/blob/main/FX_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

# supporting dependencies
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np

In [ ]:
# Authorization
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# **IMPORT DATA**

In [ ]:
def gload_CSV(link_id1, dir1, link_id2='', dir2=''):
  d1 = None
  d2 = None
  #
  d1 = drive.CreateFile({'id':link_id1})
  d1.GetContentFile(dir1)
  #
  if link_id2 != '' and dir2 != '':
    d2 = drive.CreateFile({'id':link_id2})
    d2.GetContentFile(dir2)

In [ ]:
link_id = '18QQBji5C9XBur5NpzLVcNEt_LOIc8D2S'
dir = '/content/drive/MyDrive/Colab Notebooks/Development lab/Forex Data/FX Historical Data/GBPUSD/GBPUSD15.csv'

In [ ]:
csv = gload_CSV(link_id,dir)
data = pd.read_csv(dir, names=['Date','Time','Open','High','Low','Close','Volume'])
data.head()

,Date,Time,Open,High,Low,Close,Volume
0,2019.08.19,10:30,1.21353,1.21353,1.21243,1.21278,2349
1,2019.08.19,10:45,1.21275,1.21356,1.21262,1.21334,2507
2,2019.08.19,11:00,1.21333,1.21355,1.21247,1.21255,1574
3,2019.08.19,11:15,1.21253,1.21316,1.21244,1.21275,1712
4,2019.08.19,11:30,1.21274,1.21290,1.21174,1.21200,1968


# **PRE-PROCESS DATA**

In [ ]:
data.describe()

,Open,High,Low,Close,Volume
count,65001.000000,65001.000000,65001.000000,65001.000000,65001.000000
mean,1.322074,1.322564,1.321582,1.322076,1055.919078
std,0.056102,0.056022,0.056178,0.056100,789.481062
min,1.141640,1.143510,1.140920,1.141640,1.000000
25%,1.287480,1.287980,1.286930,1.287490,483.000000
50%,1.323850,1.324360,1.323370,1.323840,880.000000
75%,1.370860,1.371290,1.370450,1.370870,1413.000000
max,1.424210,1.424790,1.423990,1.424210,9874.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65001 entries, 0 to 65000
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    65001 non-null  object 
 1   Time    65001 non-null  object 
 2   Open    65001 non-null  float64
 3   High    65001 non-null  float64
 4   Low     65001 non-null  float64
 5   Close   65001 non-null  float64
 6   Volume  65001 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 3.5+ MB


In [ ]:
data['Target'] = data.Open.shift(-3)
data.Target

0        1.21253
1        1.21274
2        1.21200
3        1.21244
4        1.21287
          ...   
64996    1.31325
64997    1.31345
64998        NaN
64999        NaN
65000        NaN
Name: Target, Length: 65001, dtype: float64

In [ ]:
data['Target'] = (data.Target - data.Open)
data.Target

0       -0.00100
1       -0.00001
2       -0.00133
3       -0.00009
4        0.00013
          ...   
64996   -0.00092
64997   -0.00060
64998        NaN
64999        NaN
65000        NaN
Name: Target, Length: 65001, dtype: float64

In [ ]:
data['Prev'] = data.Close.shift(3)
data['Prev'] = (data.Open - data.Prev)
data.Prev

0            NaN
1            NaN
2            NaN
3       -0.00025
4       -0.00060
          ...   
64996    0.00106
64997    0.00061
64998   -0.00017
64999   -0.00081
65000   -0.00056
Name: Prev, Length: 65001, dtype: float64

In [ ]:
def getAction(value): 
    if(value < -0.0005):
        return 'SELL'
    if(value > 0.0005):
        return 'BUY'
    # return hold action
    return 'HOLD'

In [ ]:
data['Action'] = [getAction(x) for x in data.Target]
data.Action.value_counts()

HOLD    30241
BUY     17534
SELL    17226
Name: Action, dtype: int64

In [ ]:
data['Volume'] = data.Volume/1000
data['Target'] = data.High - data.Low
data.head()

,Date,Time,Open,High,Low,Close,Volume,Target,Prev,Action
0,2019.08.19,10:30,1.21353,1.21353,1.21243,1.21278,2.349,0.00110,NaN,SELL
1,2019.08.19,10:45,1.21275,1.21356,1.21262,1.21334,2.507,0.00094,NaN,HOLD
2,2019.08.19,11:00,1.21333,1.21355,1.21247,1.21255,1.574,0.00108,NaN,SELL
3,2019.08.19,11:15,1.21253,1.21316,1.21244,1.21275,1.712,0.00072,-0.00025,HOLD
4,2019.08.19,11:30,1.21274,1.21290,1.21174,1.21200,1.968,0.00116,-0.00060,HOLD


In [ ]:
data = data.dropna()
data.tail()

,Date,Time,Open,High,Low,Close,Volume,Target,Prev,Action
64996,2022.03.31,04:00,1.31417,1.31439,1.31350,1.31406,1.562,0.00089,0.00106,SELL
64997,2022.03.31,04:15,1.31405,1.31425,1.31364,1.31401,0.972,0.00061,0.00061,SELL
64998,2022.03.31,04:30,1.31398,1.31402,1.31291,1.31324,1.262,0.00111,-0.00017,HOLD
64999,2022.03.31,04:45,1.31325,1.31362,1.31295,1.31329,1.088,0.00067,-0.00081,HOLD
65000,2022.03.31,05:00,1.31345,1.31345,1.31327,1.31334,0.169,0.00018,-0.00056,HOLD


# **PREPARE DATA**

In [ ]:
data_size = data.Time.size
data_size

64998

In [ ]:
action_dict = {'SELL':1,'BUY':2,'HOLD':0}
data['Action'] = data.Action.apply(lambda x: action_dict[x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
tn_set = int(0.65*data_size)
tt_set = int(0.3*data_size)
vn_set = int(0.05*data_size)
#---- 
print(tn_set,tt_set,vn_set)

42248 19499 3249


In [ ]:
train_df = data.iloc[:tn_set,2:]
train_df.head()

,Open,High,Low,Close,Volume,Target,Prev,Action
3,1.21253,1.21316,1.21244,1.21275,1.712,0.00072,-0.00025,0
4,1.21274,1.21290,1.21174,1.21200,1.968,0.00116,-0.00060,0
5,1.21200,1.21289,1.21155,1.21244,1.939,0.00134,-0.00055,0
6,1.21244,1.21295,1.21223,1.21286,1.309,0.00072,-0.00031,0
7,1.21287,1.21299,1.21234,1.21245,1.150,0.00065,0.00087,1


In [ ]:
train_df.Open.size

42248

In [ ]:
test_df = data.iloc[tn_set:(tt_set+tn_set),2:]
test_df.head()

,Open,High,Low,Close,Volume,Target,Prev,Action
42251,1.39103,1.39119,1.39063,1.39064,0.587,0.00056,0.00012,0
42252,1.39063,1.39115,1.39062,1.39100,0.568,0.00053,-0.00035,0
42253,1.39110,1.39138,1.39098,1.39114,0.861,0.00040,0.00007,0
42254,1.39115,1.39116,1.39060,1.39084,0.958,0.00056,0.00051,1
42255,1.39082,1.39134,1.39071,1.39124,0.910,0.00063,-0.00018,0


In [ ]:
test_df.Open.size

19499

In [ ]:
validation_df = data.iloc[(tt_set+tn_set):,2:9]
validation_df.head()

,Open,High,Low,Close,Volume,Target,Prev
61750,1.35286,1.35348,1.35277,1.35310,1.298,0.00071,-0.00094
61751,1.35310,1.35324,1.35193,1.35249,1.126,0.00131,-0.00020
61752,1.35250,1.35281,1.35249,1.35262,0.859,0.00032,-0.00035
61753,1.35263,1.35309,1.35250,1.35303,0.953,0.00059,-0.00047
61754,1.35303,1.35347,1.35299,1.35339,0.808,0.00048,0.00054


In [ ]:
validation_df.Open.size

3251

In [ ]:
check_ = data.iloc[(tt_set+tn_set):,7:]
check_.head()

,Target,Prev,Action
61750,0.00071,-0.00094,0
61751,0.00131,-0.00020,0
61752,0.00032,-0.00035,2
61753,0.00059,-0.00047,2
61754,0.00048,0.00054,0


In [ ]:
train_x = np.column_stack((train_df.Open.values,train_df.High.values,train_df.Low.values,train_df.Close.values,train_df.Volume.values,train_df.Target.values,train_df.Prev.values))
test_x  = np.column_stack((test_df.Open.values,test_df.High.values,test_df.Low.values,test_df.Close.values,test_df.Volume.values,test_df.Target.values,test_df.Prev.values))
valid_x  = np.column_stack((validation_df.Open.values,validation_df.High.values,validation_df.Low.values,validation_df.Close.values,validation_df.Volume.values,validation_df.Target.values,validation_df.Prev.values))

# **MODEL Design**

In [ ]:
model = None
# load model
#model = keras.models.load_model('GBPUSD15_model.h5')
# summarize model
#model.summary()

# create and compile model if not created
#if(model.dtype != 'float32'):
model = keras.Sequential([keras.layers.Dense(50, input_shape=(7,), activation='relu'),
                          keras.layers.Dense(150, activation='relu'),
                          keras.layers.Dense(150, activation='relu'),
                          keras.layers.Dense(150, activation='relu'),
                          keras.layers.Dense(150, activation='relu'),
                          keras.layers.Dense(150, activation='relu'),
                          keras.layers.Dense(250, activation='relu'),
                          keras.layers.Dense(3)])
model.compile(optimizer='adam',
             loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])
  # fit/train model
model.fit(train_x, train_df.Action.values, batch_size=10, epochs=500)

Epoch 1/500
4225/4225 [==============================] - 15s 3ms/step - loss: 1.0384 - accuracy: 0.4618
Epoch 2/500
4225/4225 [==============================] - 13s 3ms/step - loss: 1.0348 - accuracy: 0.4644
Epoch 3/500
4225/4225 [==============================] - 13s 3ms/step - loss: 1.0333 - accuracy: 0.4610
Epoch 4/500
4225/4225 [==============================] - 13s 3ms/step - loss: 1.0328 - accuracy: 0.4646
Epoch 5/500
4225/4225 [==============================] - 13s 3ms/step - loss: 1.0323 - accuracy: 0.4630
Epoch 6/500
4225/4225 [==============================] - 13s 3ms/step - loss: 1.0320 - accuracy: 0.4643
Epoch 7/500
4225/4225 [==============================] - 13s 3ms/step - loss: 1.0317 - accuracy: 0.4654
Epoch 8/500
4225/4225 [==============================] - 13s 3ms/step - loss: 1.0318 - accuracy: 0.4643
Epoch 9/500
4225/4225 [==============================] - 13s 3ms/step - loss: 1.0315 - accuracy: 0.4664
Epoch 10/500
4225/4225 [==============================] - 13s 3m

# **EVALUATE MODEL**

In [ ]:
scores = model.evaluate(test_x, test_df.Action.values)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

610/610 [==============================] - 1s 1ms/step - loss: 0.9491 - accuracy: 0.5438
accuracy: 54.38%


# **MAKE PREDICTION**

In [ ]:
confidence = pd.DataFrame(model.predict(valid_x))
confidence.head()

,0,1,2
0,0.266777,-0.134191,-0.154199
1,-0.087173,-0.028305,0.030870
2,0.830134,-0.354858,-0.426478
3,0.529500,-0.239220,-0.278989
4,0.733343,-0.323805,-0.375128


In [ ]:
check_.head()

,Target,Prev,Action
61750,0.00071,-0.00094,0
61751,0.00131,-0.00020,0
61752,0.00032,-0.00035,2
61753,0.00059,-0.00047,2
61754,0.00048,0.00054,0


In [ ]:
prediction = [confidence.iloc[x].idxmax() for x in confidence.index]
ptable = np.column_stack((prediction,check_.Action.values))

In [ ]:
ptable = pd.DataFrame(ptable,columns=['Predicted','Expected'])
ptable.Predicted = ptable.Predicted.replace(np.nan,)
ptable['diff'] = ptable.Predicted - ptable.Expected
print(ptable.head())
print("  ")
print(ptable.iloc[:,2].value_counts())

   Predicted  Expected  diff
0          0         0     0
1          2         0     2
2          0         2    -2
3          0         2    -2
4          0         0     0
  
 0    1486
-1     560
-2     511
 1     417
 2     277
Name: diff, dtype: int64


# **Serialize model to JSON**

In [ ]:
model_json = model.to_json()
with open("GBPUSD15_model.json","w") as json_file:
  json_file.write(model_json)

# **Serialize weights to HDF5**

In [ ]:
model.save_weights("GBPUSD15_model.h5")
print("Saved model to disk")

Saved model to disk


# **Load json and create model**

In [ ]:
json_file = open('GBPUSD15_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)

# **Load weights into new model**

In [ ]:
loaded_model.load_weights("GBPUSD15_model.h5")
print("Loaded model from disk")

Loaded model from disk


# **Evaluate loaded model on test data**

In [ ]:
loaded_model.compile(optimizer='adam',
             loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [ ]:
scores = loaded_model.evaluate(test_x, test_df.Action.values)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], scores[1]*100))

610/610 [==============================] - 1s 1ms/step - loss: 0.9491 - accuracy: 0.5438
accuracy: 54.38%


# **SAVE NEURAL NETWORK MODEL AS KERAS MODEL**

In [ ]:
# save model and architecture to single file
model.save("GBPUSD15_model.h5")
print("Saved model to disk")

Saved model to disk


### **APPLICATION**

In [ ]:
 open  =  
high  = 
low   =  
close =  

SyntaxError: ignored